In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data   #这一行是引入tensorflow的mnist数据

#number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True) #这句话的意思就是假如你的电脑上没有这个数据包，那么就会自动从
#网上下载下来。第二次运行这个程序的时候就会直接从下载好的数据包中读取你的数据，不用再上网下载了

#定义添加神经层的函数
def add_layer(inputs,in_size,out_size,n_layer,activation_function=None):
    layer_name = 'layer%s' %n_layer
    with tf.name_scope('layer'):
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.random.normal([in_size,out_size]),name='w')
            tf.summary.histogram(layer_name+'/Weights',Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size])+0.1,name='b')
            tf.summary.histogram(layer_name+'/biases',biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
            tf.summary.histogram(layer_name+'/outputs',outputs)
            return outputs
        
        
#定义所计算的精度函数，就是用来对比的
def compute_accuracy(v_xs,v_ys):
    global prediction   #这一行就是把 prediction 定义为全局变量
    y_pre = sess.run(prediction,feed_dict={xs:v_xs})  #把xs feed到prediction中去，生成预测值。 
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1)) #把生成的预测值与真实值作比较，看看输出的1的位置是否相同
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #记录在对比当中有多少个是对的，有多少个是错的
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys})
    return result #这里输出的是一个百分比,预测准确度
        
#定义placeholder给神经网络传递数据
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None,784],name='x_input') #这里的None代表不限制样本数量，784就是代表一张图片的像素
    ys = tf.placeholder(tf.float32,[None,10],name='y_input') #这里的10 就是代表有10个输出
    
#添加输出层
prediction = add_layer(xs,784,10,n_layer=1,activation_function=tf.nn.softmax) #所输入的一个图片有多大，然后就是有10个输出
                                                                              #使用softmax激活函数
    
#计算代价函数，在分类当中使用误差方法就不是平方误差了，改用一下对应的方法
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))   
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


        
#对于使用tensorboard最重要的步骤
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('kuangtu'.sess.graph)


for i in range(1000):
    batch_xs,batch_ys=mnist.train.next_batch(100) #这里就是提取数据集中的100个样本作为训练集
    sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys})
    if i % 50 == 0:
        print(compute_accuracy(mnist.test.images,mnist.test.labels))  #这个是定义的精度函数，用测试集来检测
                                                                      #输出 预测准确度， 是一个百分比
        
        result=sess.run(merged)
        writer=tf.add_summary(result,i)